In [8]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error

data = pd.read_csv("./data/train_data.csv")
data = data.iloc[:, 1:]
data.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,Action,time,event
0,0.62,0.03,0.10,0.28,0.43,0.67,0.38,0.12,0.15,0.88,0.72,0.89,0.43,0.36,0.43,0.66,0.16,0,0.47,1
1,0.12,0.87,0.43,0.28,0.02,0.23,0.82,0.14,0.81,0.68,0.53,0.24,0.05,0.01,0.96,0.98,0.34,0,1.39,1
2,0.37,0.06,0.47,0.86,0.86,0.33,0.01,0.90,0.79,0.43,0.64,0.84,0.05,0.56,0.01,0.77,0.62,0,3.61,1
3,0.57,0.39,0.65,0.76,0.83,0.92,0.57,0.60,0.34,0.35,0.44,0.93,0.44,0.65,0.61,0.25,0.96,0,1.02,1
4,0.82,0.00,0.26,0.27,0.82,0.68,0.43,0.92,0.81,0.26,0.66,0.04,0.10,0.93,0.82,0.37,0.09,1,2.07,1


In [9]:
X = data.iloc[:, :18]
y = data.iloc[:, 18]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [15]:
X_train.iloc[:, 17].shape

(532,)

In [12]:
y_train.shape

(532,)

In [36]:
for i in range(18):
    dataset = lgb.Dataset(X_train.iloc[:, i].to_numpy().reshape(-1, 1), label=y_train.to_numpy())
    
    params = {'learning_rate': 0.0001, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}
    
    regressor = lgb.train(params, dataset, 10000)
    pred = regressor.predict(X_val)
    error = mean_absolute_error(y_val, pred)
    
    print(f"{i} - {error.round(2)}")
    

0 - 7.37
1 - 7.32
2 - 7.5
3 - 7.27
4 - 7.33
5 - 7.09
6 - 7.29
7 - 7.21
8 - 7.31
9 - 7.77
10 - 7.41
11 - 7.35
12 - 7.49
13 - 7.06
14 - 7.38
15 - 7.24
16 - 7.18
17 - 7.48


In [37]:
dataset = lgb.Dataset(X_train.iloc[:, [5, 12]], label=y_train)

params = {'learning_rate': 0.00001, 
      'max_depth': 50, 
      'boosting': 'gbdt', 
      'objective': 'regression', 
      'metric': 'mse', 
      'is_training_metric': True, 
      'num_leaves': 250, 
      'feature_fraction': 0.9, 
      'bagging_fraction': 0.7, 
      'bagging_freq': 30, 
      'seed':2018}

regressor = lgb.train(params, dataset, 10000)
pred = regressor.predict(X_val)

In [38]:
from sklearn.metrics import mean_squared_error
import numpy as np

# np.sqrt(mean_squared_error(y_val, pred))
mean_absolute_error(y_val, pred)

7.210563860429485

In [41]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_val)

np.sqrt(mean_squared_error(y_val, pred))

1736.8557179809625

In [68]:
test_data = pd.read_csv('./data/testX.csv')
test_data = test_data.iloc[:, 1:]

pred = clf.predict(test_data.to_numpy())

In [69]:
result = pd.DataFrame(zip(list(range(287)), map(int, pred.tolist())), columns=['Title', 'action'])
result.to_csv('./result/tabnet_result.csv', index=False)
result

,Title,action
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
281,281,1
282,282,0
283,283,0
284,284,0
